In [120]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

import os

In [103]:
# Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside the Dummy Environment
env = DummyVecEnv([lambda : env])
# Stack the frames
env = VecFrameStack(env, n_stack=4, channels_order="last")

In [104]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [105]:
CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"

In [106]:
callback = TrainAndLoggingCallback(check_freq = 1e3, save_path = CHECKPOINT_DIR)

In [123]:
# Creat the AI model
model = PPO("CnnPolicy", env, verbose=1)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [125]:
# Train the AI model
model.learn(total_timesteps=1, callback=callback)

-----------------------------
| time/              |      |
|    fps             | 74   |
|    iterations      | 1    |
|    time_elapsed    | 27   |
|    total_timesteps | 2048 |
-----------------------------


In [126]:
# Save model
model.save("ppo_mario")

In [127]:
# Load model
model = PPO.load("ppo_mario")


In [128]:
# Test the RL model
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

KeyboardInterrupt: 

In [129]:
env.close()